In [1]:
import numpy as np

class QuantumState:
    def __init__(self, num_qubits):
        self.num_qubits = num_qubits
        self.state = np.zeros(2 ** num_qubits, dtype=np.complex128)
        self.state[0] = 1.0  # Initialize to |00...0>

    def apply_x(self, target):
        """Pauli-X (NOT) gate"""
        new_state = np.zeros_like(self.state)
        for idx in range(len(self.state)):
            new_idx = idx ^ (1 << target)
            new_state[new_idx] += self.state[idx]
        self.state = new_state

    def apply_h(self, target):
        """Hadamard gate"""
        new_state = np.zeros_like(self.state)
        sqrt2 = np.sqrt(2)
        for idx in range(len(self.state)):
            bit = (idx >> target) & 1
            masked_idx = idx & ~(1 << target)
            if bit == 0:
                new_state[masked_idx] += self.state[idx] / sqrt2
                new_state[masked_idx | (1 << target)] += self.state[idx] / sqrt2
            else:
                new_state[masked_idx] += self.state[idx] / sqrt2
                new_state[masked_idx | (1 << target)] -= self.state[idx] / sqrt2
        self.state = new_state

    def apply_z(self, target):
        """Pauli-Z gate"""
        for idx in range(len(self.state)):
            if (idx >> target) & 1:
                self.state[idx] *= -1

    def apply_cnot(self, control, target):
        """Controlled-NOT gate"""
        new_state = np.zeros_like(self.state)
        for idx in range(len(self.state)):
            control_bit = (idx >> control) & 1
            if control_bit:
                new_idx = idx ^ (1 << target)
                new_state[new_idx] += self.state[idx]
            else:
                new_state[idx] += self.state[idx]
        self.state = new_state

    def get_probabilities(self):
        """Return measurement probabilities"""
        return np.abs(self.state) ** 2

    def measure_all(self):
        """Collapse the state to a classical basis state"""
        probabilities = self.get_probabilities()
        outcome = np.random.choice(len(probabilities), p=probabilities)
        self.state = np.zeros_like(self.state)
        self.state[outcome] = 1.0
        return outcome

    def __str__(self):
        return str(self.state)

In [8]:
if __name__ == "__main__":
    qc = QuantumState(2)
    print("Initial state:")
    print(qc.state)
    
    qc.apply_h(0)
    qc.apply_cnot(0, 1)
    
    print("\nState after H(0) and CNOT(0→1):")
    print(qc.state)
    
    circuit_state = qc.state.copy()
    
    print("\nMeasurement probabilities:")
    print(qc.get_probabilities())
    
    results = []
    for _ in range(1000):
        qc.state = circuit_state.copy()
        outcome = qc.measure_all()
        results.append(outcome)
    
    print("\nMeasurement results (0 for |00>, 1 for |01>, 2 for |10>, 3 for |11>):")
    print(np.bincount(results))
    
    print("\nFinal state after last measurement:")
    print(qc.state)
    
    print("\nFinal probabilities after last measurement:")
    print(qc.get_probabilities())
    

Initial state:
[1.+0.j 0.+0.j 0.+0.j 0.+0.j]

State after H(0) and CNOT(0→1):
[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]

Measurement probabilities:
[0.5 0.  0.  0.5]

Measurement results (0 for |00>, 1 for |01>, 2 for |10>, 3 for |11>):
[490   0   0 510]

Final state after last measurement:
[0.+0.j 0.+0.j 0.+0.j 1.+0.j]

Final probabilities after last measurement:
[0. 0. 0. 1.]
